In [1]:
import os
import tensorflow as tf

os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
from glob import glob
from PIL import Image, ImageOps
import matplotlib.pyplot as plt

import keras
from keras import layers

import requests
import zipfile
import os



In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 16
MAX_TRAIN_IMAGES = 350


def data_build(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(images=image, size=[IMAGE_SIZE, IMAGE_SIZE])
    image = image / 255.0
    return image


def preprocessing(low_light_images):
    dataset = tf.data.Dataset.from_tensor_slices((low_light_images))
    dataset = dataset.map(data_build, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    return dataset


trainig_data = sorted(glob("./Train/low/*"))[:MAX_TRAIN_IMAGES]
valid_data = sorted(glob("./Train/low/*"))[MAX_TRAIN_IMAGES: 417]
test_data = sorted(glob("./Train/low/*"))[418 :]
#test_low_light_images = sorted(glob("./lol_dataset/eval15/low/*"))


train= preprocessing(trainig_data)
val_dataset = preprocessing(valid_data)

print("Train Dataset:", train)
print("Validation Dataset:", val_dataset)

Train Dataset: <_BatchDataset element_spec=TensorSpec(shape=(16, 256, 256, 3), dtype=tf.float32, name=None)>
Validation Dataset: <_BatchDataset element_spec=TensorSpec(shape=(16, 256, 256, 3), dtype=tf.float32, name=None)>


In [3]:
def lossanalysis(X, num_clusters=3):
    kmeans = KMeans(n_clusters=num_clusters)
    cluster_labels = kmeans.fit_predict(X)
    return cluster_labels, kmeans.cluster_centers_

def losspredict():
    data = request.get_json(force=True)
    features = np.array(data['features']).reshape(1, -1)
    prediction = model.predict(features)
    return jsonify({'prediction': prediction.tolist()})

def lossoptimization(estimator, param_bounds, X_train, y_train, cv=5):
    def objective_function(**params):
        estimator.set_params(**params)
        scores = cross_val_score(estimator, X_train, y_train, cv=cv, scoring='accuracy')
        return scores.mean()

    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds=param_bounds,
        random_state=42,
    )
    optimizer.maximize(init_points=5, n_iter=10)
    best_params = optimizer.max['params']
    return best_params

In [4]:
def basic_model():
    image = keras.Input(shape=[None, None, 3])
    a1 = layers.Conv2D(
        32, (3, 3), strides=(1, 1), activation="relu", padding="same"
    )(image)
    a2 = layers.Conv2D(
        32, (3, 3), strides=(1, 1), activation="relu", padding="same"
    )(a1)
    a3 = layers.Conv2D(
        32, (3, 3), strides=(1, 1), activation="relu", padding="same"
    )(a2)
    a4 = layers.Conv2D(
        32, (3, 3), strides=(1, 1), activation="relu", padding="same"
    )(a3)
    int_con1 = layers.Concatenate(axis=-1)([a4, a3])
    a5 = layers.Conv2D(
        32, (3, 3), strides=(1, 1), activation="relu", padding="same"
    )(int_con1)
    int_con2 = layers.Concatenate(axis=-1)([a5, a2])
    a6 = layers.Conv2D(
        32, (3, 3), strides=(1, 1), activation="relu", padding="same"
    )(int_con2)
    int_con3 = layers.Concatenate(axis=-1)([a6, a1])
    result = layers.Conv2D(24, (3, 3), strides=(1, 1), activation="tanh", padding="same")(
        int_con3
    )
    return keras.Model(inputs=image, outputs=result)

In [5]:
def colorloss(x):
    mean_score = tf.reduce_mean(x, axis=(1, 2), keepdims=True)
    mean_red, mean_green, mean_blue = (
        mean_score[:, :, :, 0],
        mean_score[:, :, :, 1],
        mean_score[:, :, :, 2],
    )
    delta_red_green = tf.square(mean_red - mean_green)
    delta_green_blue = tf.square(mean_blue - mean_green)
    delta_blue_red =  tf.square(mean_blue - mean_red)
    return tf.sqrt(tf.square(delta_red_green) + tf.square(delta_blue_red) + tf.square(delta_green_blue))

In [6]:
def lossbalance(X, y):
    print(f'Original class distribution: {Counter(y)}')
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    print(f'Resampled class distribution: {Counter(y_resampled)}')
    return X_resampled, y_resampled

def ilosscheck(X_train, y_train):
    estimators = [
        ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
        ('svc', SVC(random_state=42))
    ]
    stack_model = StackingClassifier(
        estimators=estimators,
        final_estimator=LogisticRegression()
    )
    stack_model.fit(X_train, y_train)
    return stack_model

In [7]:
def exposure_loss(x, mean_val=0.6):
    x = tf.reduce_mean(x, axis=3, keepdims=True)
    mean = tf.nn.avg_pool2d(x, ksize=16, strides=16, padding="VALID")
    return tf.reduce_mean(tf.square(mean - mean_val))

In [8]:
def lossinsmoothness(x):
    batch_size = tf.shape(x)[0]
    height = tf.shape(x)[1]
    width = tf.shape(x)[2]
    height_count = (tf.shape(x)[2] - 1) * tf.shape(x)[3]
    width_count = tf.shape(x)[2] * (tf.shape(x)[3] - 1)
    hori_variation = tf.reduce_sum(tf.square((x[:, 1:, :, :] - x[:, : height - 1, :, :])))
    verti_variation = tf.reduce_sum(tf.square((x[:, :, 1:, :] - x[:, :, : width - 1, :])))
    batch_size = tf.cast(batch_size, dtype=tf.float32)
    height_count = tf.cast(height_count, dtype=tf.float32)
    width_count = tf.cast(width_count, dtype=tf.float32)
    return 2 * (hori_variation / height_count + verti_variation / width_count) / batch_size

In [9]:
def losscount(model, X, index):
    explainer = lime.lime_tabular.LimeTabularExplainer(X, mode='regression')
    explanation = explainer.explain_instance(X[index], model.predict, num_features=len(X[index]))
    return explanation

def smoothness(series, order=(1, 1, 1)):
    model = ARIMA(series, order=order)
    fitted_model = model.fit()
    return fitted_model

def ikcolor(model, steps=10):
    forecast = model.forecast(steps=steps)
    return forecast

In [10]:
class SpatialConsistencyLoss(keras.losses.Loss):
    def __init__(self, **kwargs):
        super().__init__(reduction="none")

        self.left_kernel = tf.constant(
            [[[[0, 0, 0]], [[-1, 1, 0]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.right_kernel = tf.constant(
            [[[[0, 0, 0]], [[0, 1, -1]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.up_kernel = tf.constant(
            [[[[0, -1, 0]], [[0, 1, 0]], [[0, 0, 0]]]], dtype=tf.float32
        )
        self.down_kernel = tf.constant(
            [[[[0, 0, 0]], [[0, 1, 0]], [[0, -1, 0]]]], dtype=tf.float32
        )

    def call(self, true_image, predicted_image):
        original_mean = tf.reduce_mean(true_image, 3, keepdims=True)
        enhanced_mean = tf.reduce_mean(predicted_image, 3, keepdims=True)
        original_pool = tf.nn.avg_pool2d(
            original_mean, ksize=4, strides=4, padding="VALID"
        )
        enhanced_pool = tf.nn.avg_pool2d(
            enhanced_mean, ksize=4, strides=4, padding="VALID"
        )

        original_left = tf.nn.conv2d(
            original_pool,
            self.left_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )
        original_right = tf.nn.conv2d(
            original_pool,
            self.right_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )
        original_up = tf.nn.conv2d(
            original_pool, self.up_kernel, strides=[1, 1, 1, 1], padding="SAME"
        )
        original_down = tf.nn.conv2d(
            original_pool,
            self.down_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )

        enhanced_left = tf.nn.conv2d(
            enhanced_pool,
            self.left_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )
        enhanced_right = tf.nn.conv2d(
            enhanced_pool,
            self.right_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )
        enhanced_up = tf.nn.conv2d(
            enhanced_pool, self.up_kernel, strides=[1, 1, 1, 1], padding="SAME"
        )
        enhanced_down = tf.nn.conv2d(
            enhanced_pool,
            self.down_kernel,
            strides=[1, 1, 1, 1],
            padding="SAME",
        )

        left = tf.square(original_left - enhanced_left)
        right = tf.square(original_right - enhanced_right)
        up = tf.square(original_up - enhanced_up)
        down = tf.square(original_down - enhanced_down)
        return left + right + up + down

In [11]:
def feature(X_train, y_train):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]

    plt.figure(figsize=(10, 6))
    plt.title("Feature importance")
    plt.bar(range(X_train.shape[1]), importances[indices],
            color="b", align="center")
    plt.xticks(range(X_train.shape[1]), indices)
    plt.xlim([-1, X_train.shape[1]])
    plt.show()

def ikdata(X):
    imputer = IterativeImputer(random_state=0)
    X_imputed = imputer.fit_transform(X)
    return X_imputed

In [12]:
class themodel(keras.Model):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.dce_model = basic_model()

    def compile(self, learning_rate, **kwargs):
        super().compile(**kwargs)
        self.optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
        self.spatial_constancy_loss = SpatialConsistencyLoss(reduction="none")
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.lossinsmoothness_tracker = keras.metrics.Mean(
            name="lossinsmoothness"
        )
        self.spatial_constancy_loss_tracker = keras.metrics.Mean(
            name="spatial_constancy_loss"
        )
        self.colorloss_tracker = keras.metrics.Mean(
            name="colorloss"
        )
        self.exposure_loss_tracker = keras.metrics.Mean(name="exposure_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.lossinsmoothness_tracker,
            self.spatial_constancy_loss_tracker,
            self.colorloss_tracker,
            self.exposure_loss_tracker,
        ]

    def get_enhanced_image(self, data, output):
        enhancement_layer1 = output[:, :, :, :3]
        enhancement_layer2 = output[:, :, :, 3:6]
        enhancement_layer3 = output[:, :, :, 6:9]
        enhancement_layer4 = output[:, :, :, 9:12]
        enhancement_layer5 = output[:, :, :, 12:15]
        enhancement_layer6 = output[:, :, :, 15:18]
        enhancement_layer7 = output[:, :, :, 18:21]
        enhancement_layer8 = output[:, :, :, 21:24]
        intermediate_img = data + enhancement_layer1 * (tf.square(data) - data)
        intermediate_img = intermediate_img + enhancement_layer2 * (tf.square(intermediate_img) - intermediate_img)
        intermediate_img = intermediate_img + enhancement_layer3 * (tf.square(intermediate_img) - intermediate_img)
        enhanced_image = intermediate_img + enhancement_layer4 * (tf.square(intermediate_img) - intermediate_img)
        intermediate_img = enhanced_image + enhancement_layer5 * (tf.square(enhanced_image) - enhanced_image)
        intermediate_img = intermediate_img + enhancement_layer6 * (tf.square(intermediate_img) - intermediate_img)
        intermediate_img = intermediate_img + enhancement_layer7 * (tf.square(intermediate_img) - intermediate_img)
        enhanced_image = intermediate_img + enhancement_layer8 * (tf.square(intermediate_img) - intermediate_img)
        return enhanced_image

    def call(self, data):
        dce_net_output = self.dce_model(data)
        return self.get_enhanced_image(data, dce_net_output)

    def compute_losses(self, data, output):
        enhanced_image = self.get_enhanced_image(data, output)
        loss_illumination = 200 * lossinsmoothness(output)
        loss_spatial_constancy = tf.reduce_mean(
            self.spatial_constancy_loss(enhanced_image, data)
        )
        loss_color_constancy = 5 * tf.reduce_mean(colorloss(enhanced_image))
        loss_exposure = 10 * tf.reduce_mean(exposure_loss(enhanced_image))
        total_loss = (
            loss_illumination
            + loss_spatial_constancy
            + loss_color_constancy
            + loss_exposure
        )

        return {
            "total_loss": total_loss,
            "lossinsmoothness": loss_illumination,
            "spatial_constancy_loss": loss_spatial_constancy,
            "colorloss": loss_color_constancy,
            "exposure_loss": loss_exposure,
        }

    def train_step(self, data):
        with tf.GradientTape() as tape:
            output = self.dce_model(data)
            losses = self.compute_losses(data, output)

        gradients = tape.gradient(
            losses["total_loss"], self.dce_model.trainable_weights
        )
        self.optimizer.apply_gradients(zip(gradients, self.dce_model.trainable_weights))

        self.total_loss_tracker.update_state(losses["total_loss"])
        self.lossinsmoothness_tracker.update_state(
            losses["lossinsmoothness"]
        )
        self.spatial_constancy_loss_tracker.update_state(
            losses["spatial_constancy_loss"]
        )
        self.colorloss_tracker.update_state(losses["colorloss"])
        self.exposure_loss_tracker.update_state(losses["exposure_loss"])

        return {metric.name: metric.result() for metric in self.metrics}

    def test_step(self, data):
        output = self.dce_model(data)
        losses = self.compute_losses(data, output)

        self.total_loss_tracker.update_state(losses["total_loss"])
        self.lossinsmoothness_tracker.update_state(
            losses["lossinsmoothness"]
        )
        self.spatial_constancy_loss_tracker.update_state(
            losses["spatial_constancy_loss"]
        )
        self.colorloss_tracker.update_state(losses["colorloss"])
        self.exposure_loss_tracker.update_state(losses["exposure_loss"])

        return {metric.name: metric.result() for metric in self.metrics}

In [13]:
def save_model(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

def load_model(filename):
    with open(filename, 'rb') as f:
        model = pickle.load(f)
    return model



In [14]:
def psnr(true_image, predicted_image):
    return tf.image.psnr(true_image, predicted_image, max_val=1.0)

zero_dce_model = themodel()
zero_dce_model.compile(learning_rate=1e-4, metrics=[psnr])
history = zero_dce_model.fit(train, validation_data=val_dataset, epochs=100)
def plot_result(obj_pic):
    plt.plot(history.history[obj_pic], label=obj_pic)
    plt.plot(history.history["val_" + obj_pic], label="val_" + obj_pic)
    plt.xlabel("Epochs")
    plt.ylabel(obj_pic)
    plt.title("Train and Validation {} Over Epochs".format(obj_pic), fontsize=12)
    plt.legend()
    plt.grid()
    plt.show()


plot_result("total_loss")
plot_result("lossinsmoothness")
plot_result("spatial_constancy_loss")
plot_result("colorloss")
plot_result("exposure_loss")


Epoch 1/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 173s 8s/step - colorloss: 0.0046 - exposure_loss: 3.0131 - lossinsmoothness: 2.9645 - spatial_constancy_loss: 7.6098e-06 - total_loss: 5.9822 - val_colorloss: 0.0016 - val_exposure_loss: 2.9310 - val_lossinsmoothness: 4.0599 - val_spatial_constancy_loss: 2.6613e-06 - val_total_loss: 6.9925
Epoch 2/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 170s 8s/step - colorloss: 0.0038 - exposure_loss: 2.9985 - lossinsmoothness: 1.8824 - spatial_constancy_loss: 3.3764e-06 - total_loss: 4.8847 - val_colorloss: 0.0016 - val_exposure_loss: 2.9202 - val_lossinsmoothness: 2.9802 - val_spatial_constancy_loss: 7.8729e-06 - val_total_loss: 5.9019
Epoch 3/100
21/21 ━━━━━━━━━━━━━━━━━━━━ 207s 10s/step - colorloss: 0.0037 - exposure_loss: 2.9891 - lossinsmoothness: 1.3487 - spatial_constancy_loss: 9.5205e-06 - total_loss: 4.3415 - val_colorloss: 0.0016 - val_exposure_loss: 2.9098 - val_lossinsmoothness: 2.2869 - val_spatial_constancy_loss: 1.9065e-05 - val_total_loss: 5.1983
Epoch 4/1

KeyboardInterrupt: 

In [ ]:
def plot_results(images, titles, figure_size=(12, 12)):
    fig = plt.figure(figsize=figure_size)
    for i in range(len(images)):
        fig.add_subplot(1, len(images), i + 1).set_title(titles[i])
        _ = plt.imshow(images[i])
        plt.axis("off")
    plt.show()


def infer(original_image):
    image = keras.utils.img_to_array(original_image)
    image = image.astype("float32") / 255.0
    image = np.expand_dims(image, axis=0)
    output_image = zero_dce_model(image)
    output_image = tf.cast((output_image[0, :, :, :] * 255), dtype=np.uint8)
    output_image = Image.fromarray(output_image.numpy())
    return output_image

In [ ]:
def ensemble_with_weights(models, weights, X_train, y_train):
    estimators = [(f'model_{i}', model) for i, model in enumerate(models)]
    ensemble_model = VotingClassifier(estimators=estimators, voting='soft', weights=weights)
    ensemble_model.fit(X_train, y_train)
    return ensemble_model

def classification(X_train, y_train):
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('svm', SVC(kernel='linear'))
    ])
    pipeline.fit(X_train, y_train)
    return pipeline

In [ ]:
from PIL import Image
import numpy as np
import math

def calculate_mae(original_image, enhanced_image):
    # Mean Absolute Error calculation
    original_array = np.array(original_image)
    output_array = np.array(enhanced_image)
    return np.mean(np.abs(original_array - output_array))

def calculate_mse(original_image, enhanced_image):
    # Mean Squared Error calculation
    original_array = np.array(original_image)
    output_array = np.array(enhanced_image)
    return np.mean((original_array - output_array) ** 2)

def calculate_psnr(original_image, enhanced_image):
    # Peak Signal-to-Noise Ratio (PSNR) calculation assuming pixel values range from 0 to 255
    mse = calculate_mse(original_image, enhanced_image)
    if mse == 0:
        return float('inf')
    pixel_max = 255.0
    return 20 * math.log10(pixel_max / math.sqrt(mse))

In [ ]:
def detect_loss_svm(X):
    svm = OneClassSVM(kernel='rbf', nu=0.01)
    anomalies = svm.fit_predict(X)
    return anomalies


In [ ]:
test= preprocessing(test_data)
print("Train Dataset:", test)

In [ ]:
from PIL import Image, ImageOps
import numpy as np
import os
import cv2
from IPython.display import Image as IPImage
from IPython.display import display

total_mae = 0.0
total_mse = 0.0
total_psnr = 0.0


for idx, val_image_file in enumerate(test_data):
    original_image = Image.open(val_image_file)
    enhanced_image = infer(original_image)
    plot_results(
        [original_image,enhanced_image],
        ["Provided", "Enhanced"],
        (20, 12),
    )
    mae = calculate_mae(original_image,enhanced_image)
    mse = calculate_mse(original_image, enhanced_image)
    psnr = calculate_psnr(original_image, enhanced_image)
    total_mae += mae
    total_mse += mse
    total_psnr += psnr
    cwd = os.getcwd()
    save_dir = os.path.join(cwd, 'predicted')
    os.makedirs(save_dir, exist_ok=True)
    base_filename = os.path.splitext(os.path.basename(val_image_file))[0]
    cv2_output_filename = f"{base_filename}_cv2_{idx}.jpg"
    pil_output_filename = f"{base_filename}_pil_{idx}.jpg"
    outputphoto_np = np.array(enhanced_image)
    cv2.imwrite(os.path.join(save_dir, cv2_output_filename), cv2.cvtColor(outputphoto_np, cv2.COLOR_RGB2BGR))
    outputphoto_pil = Image.fromarray(outputphoto_np)
    outputphoto_pil.save(os.path.join(save_dir, pil_output_filename))
    print(f"Mean Absolute Error (MAE): {mae}")
    print(f"Mean Squared Error (MSE): {mse}")
    print(f"Peak Signal-to-Noise Ratio (PSNR): {psnr} dB")


In [ ]:
num_images = len(test_data)
avg_mae = total_mae / num_images
avg_mse = total_mse / num_images
avg_psnr = total_psnr / num_images

print(f"Average MAE across {num_images} images: {avg_mae}")
print(f"Average MSE across {num_images} images: {avg_mse}")
print(f"Average PSNR across {num_images} images: {avg_psnr} dB")